**Step 1:- Download the airpollution data**

In [ ]:
!wget -N https://raw.githubusercontent.com/UdayLab/intro2BigData/main/class_7/aeros_PM25Data.csv

--2024-01-22 03:44:35--  https://raw.githubusercontent.com/UdayLab/intro2BigData/main/class_7/aeros_PM25Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8507 (8.3K) [text/plain]
Saving to: ‘aeros_PM25Data.csv’

aeros_PM25Data.csv  100%[===================>]   8.31K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-01-22 03:44:35 (39.2 MB/s) - ‘aeros_PM25Data.csv’ saved [8507/8507]



**Step 2:- Read the data in the CSV into a dataframe**

In [ ]:
#importing pandas module
import pandas as pd
# Step 1: Reading the aeros_PM25Data into dataframe
df=pd.read_csv("aeros_PM25Data.csv")

df.head() # Printing the partial contents of the dataframe

,Unnamed: 0,TimeStamp,(140.7468006 41.8188869),(140.4396435 40.5942053),(141.7627117 40.1916885),(140.8733429 38.2932172),(140.957261 37.6422006),(140.5346664 36.3965854),(140.4021967 36.555081),(139.9744275 35.9510748),...,(135.790301 34.509211),(135.7079001 34.6916695),(133.7758944 34.6007931),(133.9318657 34.4600964),(134.2940462 34.7473902),(132.5621196 34.233441),(132.2165637 34.1698473),(130.601994 32.507843),(130.6384926 32.5010333),(127.7235186 26.2674837)
0,0,2022-04-01 01:00:00,9.0,NaN,NaN,NaN,3.0,NaN,6.0,NaN,...,5.0,7.0,NaN,2.0,8.0,4.0,NaN,NaN,NaN,NaN
1,1,2022-04-01 02:00:00,0.0,NaN,NaN,NaN,0.0,NaN,8.0,NaN,...,4.0,8.0,NaN,2.0,6.0,3.0,NaN,NaN,NaN,NaN
2,2,2022-04-01 03:00:00,2.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,...,8.0,3.0,NaN,2.0,11.0,4.0,NaN,NaN,NaN,NaN
3,3,2022-04-01 04:00:00,4.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,8.0,3.0,NaN,1.0,11.0,5.0,NaN,NaN,NaN,NaN
4,4,2022-04-01 05:00:00,4.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,5.0,4.0,NaN,1.0,8.0,5.0,NaN,NaN,NaN,NaN


**Step 3:- Preprocessing the data Target dataframe**

In [ ]:

# Step 2: Extract TimeStamp and sensor value to create a new DataFrame
df_subset = df[['TimeStamp', '(140.957261 37.6422006)']]

# Step 3: Remove outliers (negative values and values > 200)
df_subset['(140.957261 37.6422006)'] = np.where((df_subset['(140.957261 37.6422006)'] < 0) | (df_subset['(140.957261 37.6422006)'] > 200), np.nan, df_subset['(140.957261 37.6422006)'])

# Check if DataFrame is not empty
if not df_subset.empty:
    # Step 4: Create a new column for day of the week
    df_subset['day_of_week'] = pd.to_datetime(df_subset['TimeStamp']).dt.day_name()

    # Step 5: Fill NaN values using linear interpolation
    df_subset['(140.957261 37.6422006)'] = df_subset['(140.957261 37.6422006)'].interpolate(method='linear')

    # Display the updated DataFrame
    print(df_subset)
else:
    print("DataFrame is empty after removing outliers.")


              TimeStamp  (140.957261 37.6422006) day_of_week
0   2022-04-01 01:00:00                      3.0      Friday
1   2022-04-01 02:00:00                      0.0      Friday
2   2022-04-01 03:00:00                      1.0      Friday
3   2022-04-01 04:00:00                      2.0      Friday
4   2022-04-01 05:00:00                      3.0      Friday
5   2022-04-01 06:00:00                      4.0      Friday
6   2022-04-01 07:00:00                      4.0      Friday
7   2022-04-01 08:00:00                      4.0      Friday
8   2022-04-01 09:00:00                      0.0      Friday
9   2022-04-01 10:00:00                      9.0      Friday
10  2022-04-01 11:00:00                      9.0      Friday
11  2022-04-01 12:00:00                      7.0      Friday
12  2022-04-01 13:00:00                     11.0      Friday
13  2022-04-01 14:00:00                      9.0      Friday
14  2022-04-01 15:00:00                     10.0      Friday
15  2022-04-01 16:00:00 

<ipython-input-74-831f48f1d4d9>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['(140.957261 37.6422006)'] = np.where((df_subset['(140.957261 37.6422006)'] < 0) | (df_subset['(140.957261 37.6422006)'] > 200), np.nan, df_subset['(140.957261 37.6422006)'])
<ipython-input-74-831f48f1d4d9>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['day_of_week'] = pd.to_datetime(df_subset['TimeStamp']).dt.day_name()
<ipython-input-74-831f48f1d4d9>:19: SettingWithCopyWarning: 
A value is trying to b

**Step 4:- Transformation of the Target Dataframe**

In [ ]:
# Step 5: Create a new column for parts of the day
df_subset['part_of_day'] = pd.to_datetime(df_subset['TimeStamp']).dt.hour.apply(
    lambda x: 'evening' if 18 <= x < 20 else
              'afternoon' if 12 <= x < 18 else
              'morning' if 7 <= x < 12 else
              'sunrise' if 5 <= x < 7 else
              'midnight' if 0 <= x < 5 else
              'twilight' if 20 <= x < 22 else
              'midnight'
)

# Step 6: Create a new column for type of hours (peak or non-peak)
df_subset['hour_type'] = np.where(((8 <= pd.to_datetime(df_subset['TimeStamp']).dt.hour) & (pd.to_datetime(df_subset['TimeStamp']).dt.hour < 10)) |
                                  ((16 <= pd.to_datetime(df_subset['TimeStamp']).dt.hour) & (pd.to_datetime(df_subset['TimeStamp']).dt.hour < 18)),
                                  'peak', 'non-peak')

# Step 7: Create a new column for sensor health status
df_subset['sensor_health'] = np.where(df_subset['(140.957261 37.6422006)'] < 15, 'healthy', 'unhealthy')

# Step 8: Replace sensor column with pollution level for naive bayes
df_subset['pollution_level'] = np.where(df_subset['(140.957261 37.6422006)'] < 15, 'low', 'high')

print(df_subset[['day_of_week', 'part_of_day', 'hour_type', 'sensor_health']].head(30))

   day_of_week part_of_day hour_type sensor_health
0       Friday    midnight  non-peak       healthy
1       Friday    midnight  non-peak       healthy
2       Friday    midnight  non-peak       healthy
3       Friday    midnight  non-peak       healthy
4       Friday     sunrise  non-peak       healthy
5       Friday     sunrise  non-peak       healthy
6       Friday     morning  non-peak       healthy
7       Friday     morning      peak       healthy
8       Friday     morning      peak       healthy
9       Friday     morning  non-peak       healthy
10      Friday     morning  non-peak       healthy
11      Friday   afternoon  non-peak       healthy
12      Friday   afternoon  non-peak       healthy
13      Friday   afternoon  non-peak       healthy
14      Friday   afternoon  non-peak       healthy
15      Friday   afternoon      peak       healthy
16      Friday   afternoon      peak       healthy
17      Friday     evening  non-peak       healthy
18      Friday     evening  non

<ipython-input-69-c7c356bed1bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['part_of_day'] = pd.to_datetime(df_subset['TimeStamp']).dt.hour.apply(
<ipython-input-69-c7c356bed1bc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['hour_type'] = np.where(((8 <= pd.to_datetime(df_subset['TimeStamp']).dt.hour) & (pd.to_datetime(df_subset['TimeStamp']).dt.hour < 10)) |
<ipython-input-69-c7c356bed1bc>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

**Step 5:- Applying the Naive Bayes 5-fold cross-validation and storing output data in CSV file**

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

# Assuming df_subset contains the necessary columns including 'day_of_week', 'part_of_day', 'hour_type', 'sensor_health', and 'pollution_level'

# Step 10: Use Naive Bayes 5-fold cross-validation with one-hot encoding
X = pd.get_dummies(df_subset[['day_of_week', 'part_of_day', 'hour_type', 'sensor_health']], drop_first=True)
y = df_subset['pollution_level']

model = GaussianNB()
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation with error_score='raise'
try:
    predictions = cross_val_predict(model, X, y, cv=kfold)
    accuracy = accuracy_score(y, predictions)
    print("Cross-validation Accuracy: %.2f%%" % (accuracy * 100))
except Exception as e:
    print(f"Error during cross-validation: {e}")

# Add the predictions to the DataFrame
df_subset['predicted_pollution_level'] = predictions

# Print the final DataFrame
print(df_subset)

# Save the final DataFrame to a CSV file
df_subset.to_csv('final_output_table.csv', index=False)


Cross-validation Accuracy: 100.00%
              TimeStamp  (140.957261 37.6422006) day_of_week part_of_day  \
0   2022-04-01 01:00:00                      3.0      Friday    midnight   
1   2022-04-01 02:00:00                      0.0      Friday    midnight   
2   2022-04-01 03:00:00                      1.0      Friday    midnight   
3   2022-04-01 04:00:00                      2.0      Friday    midnight   
4   2022-04-01 05:00:00                      3.0      Friday     sunrise   
5   2022-04-01 06:00:00                      4.0      Friday     sunrise   
6   2022-04-01 07:00:00                      4.0      Friday     morning   
7   2022-04-01 08:00:00                      4.0      Friday     morning   
8   2022-04-01 09:00:00                      0.0      Friday     morning   
9   2022-04-01 10:00:00                      9.0      Friday     morning   
10  2022-04-01 11:00:00                      9.0      Friday     morning   
11  2022-04-01 12:00:00                      7.0     

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
<ipython-input-70-d1cbe8c0129e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['predicted_pollution_level'] = predictions
